In [ ]:
! pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import scipy.stats as sps

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from functools import lru_cache
from Levenshtein import distance as levenshtein_distance
from collections import defaultdict
import numpy as np

# Прочтение таблиц

In [ ]:
df = pd.read_excel('/content/FEV.xlsx')
df.head()

,Rank,Read.count,Read.proportion,CDR3.nucleotide.sequence,CDR3.amino.acid.sequence,bestVGene,bestJGene
0,0,5160,0.003694,CGTGCCAGCAGCGCCCGGACTAGCGGGAGTAGGGACAATGAGCAGT...,RASSARTSGSRDNEQFF,TRBV7-3,TRBJ2-1
1,1,2317,0.001659,TGTGCCAGCAGTTACCGCGGGACAGCTTGGGAGACCCAGTACTTC,CASSYRGTAWETQYF,TRBV6-2,TRBJ2-5
2,2,2154,0.001542,TGTGCCAGCAGACCGCTTCTTGACAGAAACAATGAGCAGTTCTTC,CASRPLLDRNNEQFF,TRBV27,TRBJ2-1
3,3,2073,0.001484,TGCAGTGTCCTTTCTGGGGATTGGGGCATTCAGTACTTC,CSVLSGDWGIQYF,TRBV20-1,TRBJ2-4
4,4,1583,0.001133,TGTGCCAGCAGCCCGGGAGATCAGCCCCAGCATTTT,CASSPGDQPQHF,TRBV7-9,TRBJ1-5


In [ ]:
database = pd.read_excel('/content/vdjdb_slim.xlsx')

In [ ]:
database_exp = database.rename(columns={'cdr3': 'CDR3.amino.acid.sequence'})

In [ ]:
data_claster = df[:10000]
data_claster

,Rank,Read.count,Read.proportion,CDR3.nucleotide.sequence,CDR3.amino.acid.sequence,bestVGene,bestJGene
0,0,5160,0.003694,CGTGCCAGCAGCGCCCGGACTAGCGGGAGTAGGGACAATGAGCAGT...,RASSARTSGSRDNEQFF,TRBV7-3,TRBJ2-1
1,1,2317,0.001659,TGTGCCAGCAGTTACCGCGGGACAGCTTGGGAGACCCAGTACTTC,CASSYRGTAWETQYF,TRBV6-2,TRBJ2-5
2,2,2154,0.001542,TGTGCCAGCAGACCGCTTCTTGACAGAAACAATGAGCAGTTCTTC,CASRPLLDRNNEQFF,TRBV27,TRBJ2-1
3,3,2073,0.001484,TGCAGTGTCCTTTCTGGGGATTGGGGCATTCAGTACTTC,CSVLSGDWGIQYF,TRBV20-1,TRBJ2-4
4,4,1583,0.001133,TGTGCCAGCAGCCCGGGAGATCAGCCCCAGCATTTT,CASSPGDQPQHF,TRBV7-9,TRBJ1-5
...,...,...,...,...,...,...,...
9995,10956,7,0.000005,TGTGCCACCAGTGACCTGACAGAAGAGACCCAGTACTTC,CATSDLTEETQYF,TRBV24-1,TRBJ2-5
9996,9786,7,0.000005,TGTGCCACCAGTGACGGACAGACCCACACAGATACGCAGTATTTT,CATSDGQTHTDTQYF,TRBV24-1,TRBJ2-3
9997,10388,7,0.000005,TGTGCCACCAGTGACGGACAGGGTTACAATGAGCAGTTCTTC,CATSDGQGYNEQFF,TRBV24-1,TRBJ2-1
9998,10389,7,0.000005,TGTGCCACCAGTGAGCTGGAACGGGCCGGGGAGCTGTTTTTT,CATSELERAGELFF,TRBV24-1,TRBJ2-2


# Функция кластеризации

In [ ]:
# Кэшированная функция для вычисления расстояния Левенштейна
@lru_cache(maxsize=None)
def cached_levenshtein_distance(seq1, seq2):
    return levenshtein_distance(seq1, seq2)

# Функция для кластеризации последовательностей
def cluster_sequences(sequences, max_distance=1):
    clusters = []
    for seq in sequences:
        added_to_cluster = False
        for cluster in clusters:
            if any(cached_levenshtein_distance(seq, existing_seq) <= max_distance for existing_seq in cluster):
                cluster.append(seq)
                added_to_cluster = True
                break
        if not added_to_cluster:
            clusters.append([seq])
    return clusters

# Функция для объединения кластеров
def merge_clusters(clusters, max_distance=1):
    merged_clusters = []
    for cluster in clusters:
        added_to_cluster = False
        for merged_cluster in merged_clusters:
            if any(cached_levenshtein_distance(seq, existing_seq) <= max_distance for seq in cluster for existing_seq in merged_cluster):
                merged_cluster.extend(cluster)
                added_to_cluster = True
                break
        if not added_to_cluster:
            merged_clusters.append(cluster)
    return merged_clusters

# Пример последовательностей аминокислот
sequences = data_claster['CDR3.amino.acid.sequence']

# Разделение данных на части
chunk_size = len(sequences) // 100
chunks = [sequences[i:i + chunk_size] for i in range(0, len(sequences), chunk_size)]

# Кластеризация каждой части
all_clusters = []
for chunk in chunks:
    all_clusters.extend(cluster_sequences(chunk, max_distance=1))

# Объединение кластеров
final_clusters = merge_clusters(all_clusters, max_distance=1)

# Вывод результатов
# Создание DataFrame
cluster_data = []
for i, cluster in enumerate(final_clusters):
    for seq in cluster:
        cluster_data.append({'Cluster': i + 1, 'Sequence': seq})

results_df = pd.DataFrame(cluster_data)

In [ ]:
# Создание нового DataFrame с данными из data_claster и номером кластера
cluster_assignments = {}
for i, cluster in enumerate(final_clusters):
  for seq in cluster:
    cluster_assignments[seq] = i + 1

results_df = data_claster.copy()
results_df['Cluster'] = results_df['CDR3.amino.acid.sequence'].map(cluster_assignments)

# Вывод нового DataFrame results_df
results_df

,Rank,Read.count,Read.proportion,CDR3.nucleotide.sequence,CDR3.amino.acid.sequence,bestVGene,bestJGene,Cluster
0,0,5160,0.003694,CGTGCCAGCAGCGCCCGGACTAGCGGGAGTAGGGACAATGAGCAGT...,RASSARTSGSRDNEQFF,TRBV7-3,TRBJ2-1,NaN
1,1,2317,0.001659,TGTGCCAGCAGTTACCGCGGGACAGCTTGGGAGACCCAGTACTTC,CASSYRGTAWETQYF,TRBV6-2,TRBJ2-5,NaN
2,2,2154,0.001542,TGTGCCAGCAGACCGCTTCTTGACAGAAACAATGAGCAGTTCTTC,CASRPLLDRNNEQFF,TRBV27,TRBJ2-1,NaN
3,3,2073,0.001484,TGCAGTGTCCTTTCTGGGGATTGGGGCATTCAGTACTTC,CSVLSGDWGIQYF,TRBV20-1,TRBJ2-4,NaN
4,4,1583,0.001133,TGTGCCAGCAGCCCGGGAGATCAGCCCCAGCATTTT,CASSPGDQPQHF,TRBV7-9,TRBJ1-5,NaN
...,...,...,...,...,...,...,...,...
9995,10956,7,0.000005,TGTGCCACCAGTGACCTGACAGAAGAGACCCAGTACTTC,CATSDLTEETQYF,TRBV24-1,TRBJ2-5,NaN
9996,9786,7,0.000005,TGTGCCACCAGTGACGGACAGACCCACACAGATACGCAGTATTTT,CATSDGQTHTDTQYF,TRBV24-1,TRBJ2-3,NaN
9997,10388,7,0.000005,TGTGCCACCAGTGACGGACAGGGTTACAATGAGCAGTTCTTC,CATSDGQGYNEQFF,TRBV24-1,TRBJ2-1,NaN
9998,10389,7,0.000005,TGTGCCACCAGTGAGCTGGAACGGGCCGGGGAGCTGTTTTTT,CATSELERAGELFF,TRBV24-1,TRBJ2-2,NaN


# Подсчёт Total Read Proportion

In [ ]:
# Подсчет количества последовательностей в каждом кластере
cluster_counts = {i + 1: len(cluster) for i, cluster in enumerate(final_clusters)}
# Подсчет суммарного Read.proportion для каждого кластера
cluster_proportions = {}
for index, row in data_claster.iterrows():
  seq = row['CDR3.amino.acid.sequence']
  proportion = row['Read.proportion']
  for i, cluster in enumerate(final_clusters):
    if seq in cluster:
      cluster_proportions[i + 1] = cluster_proportions.get(i + 1, 0) + proportion
      break

# Создание DataFrame с суммарными пропорциями, номерами кластеров и количеством последовательностей
proportions_data = []
for cluster_number, total_proportion in cluster_proportions.items():
  proportions_data.append({'Cluster': cluster_number,
               'Total Read Proportion': total_proportion,
               'Sequence Count': cluster_counts[cluster_number]})


# Сортировка по Sequence Count

In [ ]:
proportions_df = pd.DataFrame(proportions_data)
proportions_df = proportions_df.sort_values(by='Sequence Count', ascending=False)
proportions_df

,Cluster,Total Read Proportion,Sequence Count
1,5,0.000881,66
43,14,0.000021,23
41,700,0.000023,17
20,154,0.000047,14
3,831,0.000089,13
...,...,...,...
30,3727,0.000019,1
29,5337,0.000016,1
59,1445,0.000006,1
25,6505,0.000027,1


# Сортировка по Total Read Proportion

In [ ]:
proportions_df_sorted = proportions_df.sort_values(by='Total Read Proportion', ascending=False)
proportions_df_sorted = proportions_df_sorted.loc[proportions_df_sorted['Sequence Count'] > 1]
proportions_df_sorted

,Cluster,Total Read Proportion,Sequence Count
1,5,0.000881,66
0,428,0.000216,2
3,831,0.000089,13
6,836,0.000087,2
5,1074,0.000057,3
20,154,0.000047,14
15,1240,0.000042,5
28,2438,0.000035,7
11,753,0.000029,13
27,1378,0.000028,7


# Выбор кластера

In [ ]:
full = pd.merge(results_df, database_exp , on='CDR3.amino.acid.sequence', how='inner')

In [ ]:
full = full[['Cluster', 'CDR3.amino.acid.sequence', 'Read.proportion', 'bestVGene',	'bestJGene', 'v.segm',	'j.segm', 'mhc.class', 'species', 'antigen.species', 'antigen.gene']]
full

,Cluster,CDR3.amino.acid.sequence,Read.proportion,bestVGene,bestJGene,v.segm,j.segm,mhc.class,species,antigen.species,antigen.gene
0,NaN,CASSLVLADNEQFF,0.000329,TRBV7-8,TRBJ2-1,TRBV27*01,TRBJ2-1*01,MHCI,HomoSapiens,CMV,pp65
1,NaN,CASSLGISYEQYF,0.000210,TRBV7-2,TRBJ2-7,TRBV27*01,TRBJ2-7*01,MHCI,HomoSapiens,CMV,pp65
2,428.0,CASSPSYEQYF,0.000191,TRBV7-2,TRBJ2-7,TRBV5-6*01,TRBJ2-7*01,MHCI,HomoSapiens,EBV,BZLF1
3,NaN,CASSLGQTYEQYF,0.000190,TRBV7-2,TRBJ2-7,TRBV5-4*01,TRBJ2-7*01,MHCI,HomoSapiens,HomoSapiens,DNAJB1-PRKACA
4,5.0,CASSLTDTQYF,0.000163,TRBV7-9,TRBJ2-3,TRBV28*01,TRBJ2-3*01,MHCI,HomoSapiens,EBV,EBNA4
...,...,...,...,...,...,...,...,...,...,...,...
694,324.0,CASSLVGNEQFF,0.000005,TRBV5-1,TRBJ2-1,TRBV5-1*01,TRBJ2-1*01,MHCI,HomoSapiens,SARS-CoV-2,Nucleocapsid
695,NaN,CASSFRGDTEAFF,0.000005,TRBV5-1,TRBJ1-1,TRBV12-4*01,TRBJ1-1*01,MHCI,HomoSapiens,CMV,pp65
696,NaN,CASSVTGGYGYTF,0.000005,TRBV5-1,TRBJ1-2,TRBV9*01,TRBJ1-2*01,MHCII,HomoSapiens,SARS-CoV-2,Spike
697,NaN,CASSRGTGELFF,0.000005,TRBV5-1,TRBJ2-2,TRBV12-3*01,TRBJ2-2*01,MHCI,HomoSapiens,CMV,pp65


In [ ]:
full_vj = full.loc[(full['bestVGene'].str[:7] == full['v.segm'].str[:7]) & (full['bestJGene'].str[:7] == full['j.segm'].str[:7])]
full_vj

,Cluster,CDR3.amino.acid.sequence,Read.proportion,bestVGene,bestJGene,v.segm,j.segm,mhc.class,species,antigen.species,antigen.gene
7,NaN,CASSLEGQASSYEQYF,0.000161,TRBV5-1,TRBJ2-7,TRBV5-1*01,TRBJ2-7*01,MHCI,HomoSapiens,EBV,LMP2A
8,NaN,CASSLEGQASSYEQYF,0.000161,TRBV5-1,TRBJ2-7,TRBV5-1*01,TRBJ2-7*01,MHCI,HomoSapiens,EBV,BMLF1
10,NaN,CASSLEGQASSYEQYF,0.000128,TRBV5-1,TRBJ2-7,TRBV5-1*01,TRBJ2-7*01,MHCI,HomoSapiens,EBV,LMP2A
11,NaN,CASSLEGQASSYEQYF,0.000128,TRBV5-1,TRBJ2-7,TRBV5-1*01,TRBJ2-7*01,MHCI,HomoSapiens,EBV,BMLF1
12,NaN,CASSDGGATDTQYF,0.000110,TRBV6-4,TRBJ2-3,TRBV6-4*01,TRBJ2-3*01,MHCI,HomoSapiens,HomoSapiens,MLANA
...,...,...,...,...,...,...,...,...,...,...,...
652,NaN,CASSQALGSGANVLTF,0.000005,TRBV4-1,TRBJ2-6,TRBV4-1*01,TRBJ2-6*01,MHCI,HomoSapiens,CMV,pp65
675,NaN,CASSLDSSYEQYF,0.000005,TRBV5-1,TRBJ2-7,TRBV5-1*01,TRBJ2-7*01,MHCII,HomoSapiens,InfluenzaA,M1
677,NaN,CASSLENTGELFF,0.000005,TRBV5-1,TRBJ2-2,TRBV5-1*01,TRBJ2-2*01,MHCI,HomoSapiens,SARS-CoV-2,ORF1ab
684,NaN,CASSLGQETQYF,0.000005,TRBV5-1,TRBJ2-5,TRBV5-1*01,TRBJ2-5*01,MHCI,HomoSapiens,CMV,IE1


In [ ]:
full_vj['Cluster'].value_counts()

# Обработка для одного выбранного кластера

In [ ]:
results_df_one_cluster = results_df.loc[results_df['Cluster'] == 79]
results_df_one_cluster = results_df_one_cluster.rename(columns={'Sequence': 'CDR3.amino.acid.sequence'})
results_df_one_cluster

,Rank,Read.count,Read.proportion,CDR3.nucleotide.sequence,CDR3.amino.acid.sequence,bestVGene,bestJGene,Cluster


In [ ]:
full = pd.merge(results_df_one_cluster, database_exp , on='CDR3.amino.acid.sequence', how='inner')

In [ ]:
full = full[['Cluster', 'CDR3.amino.acid.sequence', 'Read.proportion', 'bestVGene',	'bestJGene', 'v.segm',	'j.segm', 'mhc.class', 'species', 'antigen.species', 'antigen.gene']]
full

,Cluster,CDR3.amino.acid.sequence,Read.proportion,bestVGene,bestJGene,v.segm,j.segm,mhc.class,species,antigen.species,antigen.gene


# Разборки с V и J

In [ ]:
full_vj = full.loc[(full['bestVGene'].str[:7] == full['v.segm'].str[:7]) & (full['bestJGene'].str[:7] == full['j.segm'].str[:7])]
full_vj